In [1]:
import sys
import torch
import numpy as np
from torchani.models import ANI2x
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from openmm.app import PDBFile
from openmmtorch import TorchForce

/home/haolan/anaconda3/envs/openmm/lib/python3.10/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


In [2]:
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {num_gpus}")
    current_gpu_name = torch.cuda.get_device_name(0)
    print(f"Current GPU Name: {current_gpu_name}")
    print("\nGPU Memory Usage:")
    print(f"  Allocated: {round(torch.cuda.memory_allocated(0) / 1024**3, 1)} GB")
    print(f"  Cached: {round(torch.cuda.memory_reserved(0) / 1024**3, 1)} GB")
else:
    print("PyTorch is running on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

CUDA Available: True
Number of GPUs: 1
Current GPU Name: NVIDIA GeForce RTX 5070 Laptop GPU

GPU Memory Usage:
  Allocated: 0.0 GB
  Cached: 0.0 GB

Using device: cuda


In [6]:
from openff.toolkit.topology import Molecule
molecule=Molecule.from_file('bioactive_min.sdf')
positions=molecule.conformers[0].to_openmm()
from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=molecule)
forcefield = app.ForceField('amber14/tip3p.xml')
forcefield.registerTemplateGenerator(gaff.generator)

modeller = app.Modeller(molecule.to_topology().to_openmm(), positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*unit.nanometers)

/home/haolan/anaconda3/envs/openmm/lib/python3.10/site-packages/openmmforcefields/generators/template_generators.py:657: UserWarning: Warning! Partial charge method 'am1bcc' is not designed for use on large (i.e. > 150 atoms) molecules and may crash or take hours to run on this molecule (found 151 atoms). For more, see https://docs.openforcefield.org/projects/toolkit/en/stable/faq.html#parameterizing-my-system-which-contains-a-large-molecule-is-taking-forever-whats-wrong
  molecule.assign_partial_charges(partial_charge_method="am1bcc")


In [ ]:
system_1 = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)
with open('gaff_system.xml', 'w') as f:
    f.write(mm.XmlSerializer.serialize(system_1))
with open('gaff_ligand_in_solvent.pdb', 'w') as output_pdb:
    PDBFile.writeFile(modeller.topology, modeller.positions, output_pdb)

In [2]:
with open('gaff_system.xml', 'r') as f:
    system_1 = mm.XmlSerializer.deserialize(f.read())
pdb = PDBFile('gaff_ligand_in_solvent.pdb')
system_1.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))

5

In [3]:
#NPT simulations for 10 ns
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(pdb.topology, system_1, integrator)
#
simulation.context.setPositions(pdb.positions)
integrator.setRandomNumberSeed(42)
print("Simulation is running on:", simulation.context.getPlatform().getName())
simulation.minimizeEnergy(maxIterations=1000)
# save the minimized structure for each replicate as starting-point geometry (almost identical actually...)
with open(f'min_complex.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), f)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
# NPT for 10 ns
simulation.reporters.append(app.DCDReporter(f'trajectory.dcd', 5000))
simulation.reporters.append(
                            app.StateDataReporter(
                                                    f'energy_data.csv',
                                                    5000,
                                                    step=True,
                                                    time=True, 
                                                    potentialEnergy=True,
                                                    kineticEnergy=True,
                                                    totalEnergy=True,
                                                    temperature=True,
                                                    volume=True,
                                                    density=True,
                                                    speed=True,
                                                    )
                            )
simulation.step(5000000)
simulation.reporters.clear()

Simulation is running on: CUDA
